In [1]:
#! pip install sqlalchemy

In [1]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc
from sqlalchemy import union, union_all
from sqlalchemy.sql import func, distinct, exists

In [2]:
engine = create_engine('sqlite:///../Data/Students.db')
metadata_obj = MetaData() # Объект-контейнер, содержит информацию о схеме данных
metadata_obj.reflect(bind=engine) # Отображение БД

Создание объектов таблиц

In [3]:
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")

In [4]:
conn=engine.connect() # Соединение

<img src=../Img/pencil.png style="width:190px;height:190px">
<p style="background-color:powderblue; font-size:125%; color: rebeccapurple;">Выполните задания</p>
<p>Выполните задания с помощью соедиинения таблиц в Jupyter Notebook </p>

1. Создайте объединение двух запросов, которые выдают значения полей *univ_name*, *city*, *rating* для всех университетов. Те из них, у которых рейтинг равен или выше 300, должны иметь комментарий ‘высокий’, все остальные — ‘низкий’.

2. Напишите команду, которая выдает список фамилий студентов с комментарием: 'успевает' у студентов, имеющих все положительные оценки; 'не успевает' для сдававших экзамены, но имеющих хотя бы одну неудовлетворительную оценку; 'не сдавал' — для всех остальных. В выводимом результате фамилии студентов упорядочите по алфавиту.

3. Выведите объединенный список студентов и преподавателей, живущих в Москве, с соответствующими комментариями 'студент' или 'преподаватель'.

4. Выведите объединенный список студентов и преподавателей ВГУ с соответствующими комментариями 'студент' или 'преподаватель'.

5. Для каждого города выведите названия университетов с минимальным и максимальным для данного города рейтингом. Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце. Резулььтат должен содержать название города, название университета, рейтинг. Упорядочите результат по городу.

6. Для каждого курса выведите фамилии студентов, получающих минимальные и максимальные на их курсе стипендии. Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце.

7. Для каждого курса выведите фамилии самого старшего и самого младшего студентов. Пометьте строки списка словами 'младший' и 'старший', поместив их в дополнительном столбце.

8. Напишите запрос для получения полного списка университетов вместе с количеством студентов, которые в них учатся. Для университетов, не имеющих студентов, поместите в список фразу ‘Студентов нет’. отсортируйте результат по убыванию числа студентов.

9. Напишите запрос для получения полного списка университетов вместе с фамилиями преподавателей, в них работающих. Для университетов, не имеющих преподавателей, поместите в список фразу ‘Преподавателей нет’. результат должен содержать название университета, фамилию и имя преподавателя. отсортируйте результат по фамилии и имени преподавателя.

10. Выведите полный список студентов вместе с оценками, полученными ими на экзаменах. Для студентов, не сдававших экзамены, в поле оценки поместите 0.

# Решение

In [7]:
def print_result(stmt):
    print(stmt)
    result = conn.execute(stmt).fetchall()
    print(len(result))
    result

In [8]:
st = student.alias('st')
em = exam_marks.alias('em')
u = university.alias('u')
l = lecturer.alias('l')
sl = subj_lect.alias('sl')
su = subject.alias('su')
st2 = student.alias('st2')
u2 = university.alias('u2')

In [20]:
# 1. Создайте объединение двух запросов, которые выдают значения полей univ_name, city, rating для всех университетов. 
# Те из них, у которых рейтинг равен или выше 300, должны иметь комментарий ‘высокий’, все остальные — ‘низкий’.


s1 = select(text("'Высокий' AS Rating"),u.c.univ_name, u.c.city, u.c.rating)
s1 = s1.where(u.c.rating >= 300)

s2 = select(text("'Низкий'"),u.c.univ_name, u.c.city, u.c.rating)
s2 = s2.where(u.c.rating < 300)

stmt = union(s1,s2)
stmt = stmt.order_by(text('Rating'), u.c.univ_name)


print_result(stmt)

SELECT 'Высокий' AS Rating, u.univ_name, u.city, u.rating 
FROM university AS u 
WHERE u.rating >= :rating_1 UNION SELECT 'Низкий', u.univ_name, u.city, u.rating 
FROM university AS u 
WHERE u.rating < :rating_2 ORDER BY Rating, univ_name

12


[('Высокий', 'БГУ', 'Белгород', 326),
 ('Высокий', 'ВГМА', 'Воронеж', 327),
 ('Высокий', 'МГТУ им. Баумана', 'Москва', 372),
 ('Высокий', 'МГУ', 'Москва', 606),
 ('Высокий', 'МФТИ', 'Москва', 500),
 ('Высокий', 'НГУ', 'Новосибирск', 345),
 ('Высокий', 'РГУ', 'Ростов', 416),
 ('Высокий', 'ТГУ', 'Томск', 368),
 ('Высокий', 'ФГБОУ "Политех"', 'Санкт-Петербург', 304),
 ('Высокий', 'ФинУ', 'Москва', 330)]

In [21]:
# 2. Напишите команду, которая выдает список фамилий студентов с комментарием: 'успевает' у студентов, 
#     имеющих все положительные оценки; 'не успевает' для сдававших экзамены, 
#     но имеющих хотя бы одну неудовлетворительную оценку; 'не сдавал' — для всех остальных. 
#     В выводимом результате фамилии студентов упорядочите по алфавиту.


s1 = select(em.c.student_id)
s1 = s1.where(em.c.student_id == st.c.student_id).exists() #сдавал

s2 = select(em.c.student_id)
s2 = s2.where(em.c.student_id == st.c.student_id)
s2 = s2.where(em.c.mark < 4).exists() #есть оценка меньше 4

s3 = select(em.c.student_id)
s3 = s3.where(em.c.student_id == st.c.student_id)
s3 = s3.where(em.c.mark < 3).exists() # не успевает

s4 = select(text("'Не сдавал'"),st.c.student_id, st.c.surname, st.c.name)
s4 = s4.where(~s1)

s5 = select(text("'Успевает'"),st.c.student_id, st.c.surname, st.c.name)
s5 = s5.where(~s2).where(s1)

s6 = select(text("'Не успевает'"),st.c.student_id, st.c.surname, st.c.name)
s6 = s6.where(s3)

stmt = union(s5, s6, s4)
# stmt = stmt.order_by() по фамилии


print_result(stmt)

SELECT 'Успевает', st.student_id, st.surname, st.name 
FROM student AS st 
WHERE NOT (EXISTS (SELECT em.student_id 
FROM exam_marks AS em 
WHERE em.student_id = st.student_id AND em.mark < :mark_1)) AND (EXISTS (SELECT em.student_id 
FROM exam_marks AS em 
WHERE em.student_id = st.student_id)) UNION SELECT 'Не успевает', st.student_id, st.surname, st.name 
FROM student AS st 
WHERE EXISTS (SELECT em.student_id 
FROM exam_marks AS em 
WHERE em.student_id = st.student_id AND em.mark < :mark_2) UNION SELECT 'Не сдавал', st.student_id, st.surname, st.name 
FROM student AS st 
WHERE NOT (EXISTS (SELECT em.student_id 
FROM exam_marks AS em 
WHERE em.student_id = st.student_id))

84


[('Не сдавал', 3, 'Петров', 'Петр'),
 ('Не сдавал', 56, 'Дуб', 'Елизавета'),
 ('Не сдавал', 57, 'Кудряшова', 'Вера'),
 ('Не сдавал', 58, 'Журавлева', 'Вера'),
 ('Не сдавал', 60, 'Рожкова', 'Пелагея'),
 ('Не сдавал', 61, 'Соколова', 'Ия'),
 ('Не сдавал', 66, 'Некрасова', 'Вероника'),
 ('Не сдавал', 68, 'Казакова', 'Ольга'),
 ('Не сдавал', 69, 'Шубина', 'Елена'),
 ('Не сдавал', 70, 'Миронова', 'Анна')]

In [22]:
# 3. Выведите объединенный список студентов и преподавателей, живущих в Москве, 
# с соответствующими комментариями 'студент' или 'преподаватель'.


s1 = select(text("'Студент'"),st.c.surname, st.c.name)
s1 = s1.where(st.c.city == 'Москва')

s2 = select(text("'Преподаватель'"),l.c.surname, l.c.name)
s2 = s2.where(l.c.city == 'Москва')


stmt = union(s1, s2)
# stmt = stmt.order_by() по фамилии


print_result(stmt)

SELECT 'Студент', st.surname, st.name 
FROM student AS st 
WHERE st.city = :city_1 UNION SELECT 'Преподаватель', l.surname, l.name 
FROM lecturer AS l 
WHERE l.city = :city_2

5


[('Преподаватель', 'Лагутин', 'Павел'),
 ('Преподаватель', 'Струков', 'Николай'),
 ('Студент', 'Баранов', 'Александр'),
 ('Студент', 'Миронова', 'Федор'),
 ('Студент', 'Сидоров', 'Вадим')]

In [12]:
# 4. Выведите объединенный список студентов и преподавателей ВГУ с соответствующими комментариями 'студент' или 'преподаватель'.


sq = select(u.c.univ_id)
sq = sq.where(u.c.univ_name == 'ВГУ').scalar_subquery()

s1 = select(text("'Студент' AS Status"), st.c.surname, st.c.name )
s1 = s1.where(st.c.univ_id == sq)

s2 = select(text("'Преподаватель' AS Status"), l.c.surname, l.c.name )
s2 = s2.where(l.c.univ_id == sq)

stmt = union(s1, s2)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result

SELECT 'Студент' AS Status, st.surname, st.name 
FROM student AS st 
WHERE st.univ_id = (SELECT u.univ_id 
FROM university AS u 
WHERE u.univ_name = :univ_name_1) UNION SELECT 'Преподаватель' AS Status, l.surname, l.name 
FROM lecturer AS l 
WHERE l.univ_id = (SELECT u.univ_id 
FROM university AS u 
WHERE u.univ_name = :univ_name_1)
24


[('Преподаватель', 'Алексеев', 'Елисей'),
 ('Преподаватель', 'Зайцева', 'Юлия'),
 ('Преподаватель', 'Зыкова', 'Оксана'),
 ('Преподаватель', 'Колесников', 'Борис'),
 ('Преподаватель', 'Максимова', 'Иванна'),
 ('Преподаватель', 'Николаев', 'Виктор'),
 ('Преподаватель', 'Никонов', 'Иван'),
 ('Преподаватель', 'Сорокин', 'Андрей'),
 ('Студент', 'Агафонов', 'Василий'),
 ('Студент', 'Аксенов', 'Валерий'),
 ('Студент', 'Белкин', 'Вадим'),
 ('Студент', 'Борисова', 'Марина'),
 ('Студент', 'Жданова', 'Ирина'),
 ('Студент', 'Зайцева', 'Ольга'),
 ('Студент', 'Иванов', 'Иван'),
 ('Студент', 'Кузнецов', 'Борис'),
 ('Студент', 'Лукин', 'Артем'),
 ('Студент', 'Мельникова', 'Дарья'),
 ('Студент', 'Некрасова', 'Вероника'),
 ('Студент', 'Павлов', 'Андрей'),
 ('Студент', 'Петров', 'Петр'),
 ('Студент', 'Семенова', 'Вероника'),
 ('Студент', 'Сергеева', 'Фёкла'),
 ('Студент', 'Федоров', 'Гостомысл')]

In [23]:
# 5. Для каждого города выведите названия университетов с минимальным и максимальным для данного города рейтингом. 
# Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце. 
# Результат должен содержать название города, название университета, рейтинг. Упорядочите результат по городу.


s1 = select(text("'max' AS Rating"), u.c.city, u.c.univ_name, func.max(u.c.rating))
s1 = s1.group_by( u.c.city)

s2 = select(text("'min' AS Rating"), u.c.city, u.c.univ_name, func.min(u.c.rating))
s2 = s2.group_by( u.c.city)

stmt = union(s1, s2).order_by(u.c.city)


print_result(stmt)

SELECT 'max' AS Rating, u.city, u.univ_name, max(u.rating) AS max_1 
FROM university AS u GROUP BY u.city UNION SELECT 'min' AS Rating, u.city, u.univ_name, min(u.rating) AS min_1 
FROM university AS u GROUP BY u.city ORDER BY city

18


[('max', 'Белгород', 'БГУ', 326),
 ('min', 'Белгород', 'БГУ', 326),
 ('max', 'Воронеж', 'ВГМА', 327),
 ('min', 'Воронеж', 'ВГУ', 296),
 ('max', 'Казань', 'КФУ', None),
 ('min', 'Казань', 'КФУ', None),
 ('max', 'Москва', 'МГУ', 606),
 ('min', 'Москва', 'ФинУ', 330),
 ('max', 'Новосибирск', 'НГУ', 345),
 ('min', 'Новосибирск', 'НГУ', 345)]

In [24]:
# 6. Для каждого курса выведите фамилии студентов, получающих минимальные и максимальные на их курсе стипендии. 
# Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце.


s1 = select(text("'max' AS Rating"), st.c.kurs, st.c.surname, func.max(st.c.stipend))
s1 = s1.group_by(  st.c.kurs)

s2 = select(text("'min' AS Rating"), st.c.kurs, st.c.surname, func.min(st.c.stipend))
s2 = s2.group_by( st.c.kurs)

stmt = union(s1, s2).order_by(st.c.kurs)


print_result(stmt)

SELECT 'max' AS Rating, st.kurs, st.surname, max(st.stipend) AS max_1 
FROM student AS st GROUP BY st.kurs UNION SELECT 'min' AS Rating, st.kurs, st.surname, min(st.stipend) AS min_1 
FROM student AS st GROUP BY st.kurs ORDER BY kurs

10


[('max', 1, 'Жук', 250),
 ('min', 1, 'Васильева', 0),
 ('max', 2, 'Зайцева', 250),
 ('min', 2, 'Кузнецов', 0),
 ('max', 3, 'Блохина', 250),
 ('min', 3, 'Рожкова', 0),
 ('max', 4, 'Казакова', 250),
 ('min', 4, 'Журавлева', 0),
 ('max', 5, 'Белкин', 250),
 ('min', 5, 'Кудряшова', 100)]

In [25]:
# 7. Для каждого курса выведите фамилии самого старшего и самого младшего студентов. 
# Пометьте строки списка словами 'младший' и 'старший', поместив их в дополнительном столбце.


s1 = select(text("'младший' AS Rating"), st.c.kurs, st.c.surname, func.max(func.date(st.c.birthday)))
s1 = s1.group_by(  st.c.kurs)

s2 = select(text("'старший' AS Rating"), st.c.kurs, st.c.surname, func.min(func.date(st.c.birthday)))
s2 = s2.group_by( st.c.kurs)

stmt = union(s1, s2).order_by(st.c.kurs)


print_result(stmt)

SELECT 'младший' AS Rating, st.kurs, st.surname, max(date(st.birthday)) AS max_1 
FROM student AS st GROUP BY st.kurs UNION SELECT 'старший' AS Rating, st.kurs, st.surname, min(date(st.birthday)) AS min_1 
FROM student AS st GROUP BY st.kurs ORDER BY kurs

10


[('младший', 1, 'Ордуханян', '2003-06-01'),
 ('старший', 1, 'Васильева', '1994-05-14'),
 ('младший', 2, 'Некрасова', '2004-08-23'),
 ('старший', 2, 'Ковалев', '1995-11-28'),
 ('младший', 3, 'Семенова', '2003-10-04'),
 ('старший', 3, 'Сорокина', '1993-09-28'),
 ('младший', 4, 'Журавлева', '2003-06-14'),
 ('старший', 4, 'Казакова', '1994-02-18'),
 ('младший', 5, 'Ситников', '2003-10-05'),
 ('старший', 5, 'Трофимов', '1995-03-24')]

In [26]:
# 8. Напишите запрос для получения полного списка университетов вместе с количеством студентов, которые в них учатся. 
# Для университетов, не имеющих студентов, поместите в список фразу ‘Студентов нет’. 
# отсортируйте результат по убыванию числа студентов.


sq = select(u.c.univ_id)
sq = sq.where(u.c.univ_id == st.c.univ_id).exists() # Сдавал что-то

s1 = select(u.c.univ_id, u.c.univ_name, func.count(st.c.surname), text('"Студенты есть"'))
s1 = s1.join(st)
s1 = s1.group_by(u.c.univ_id)


s2 = select(u.c.univ_id, u.c.univ_name, text('0 AS Количество'), text('"Студентов нет"'))
s2 = s2.where(~sq )


stmt = union(s1, s2)
stmt = stmt.order_by(func.count(st.c.surname).desc())


print_result(stmt)

SELECT u.univ_id, u.univ_name, count(st.surname) AS count_1, "Студенты есть" 
FROM university AS u JOIN student AS st ON u.univ_id = st.univ_id GROUP BY u.univ_id UNION SELECT u.univ_id, u.univ_name, 0 AS Количество, "Студентов нет" 
FROM university AS u 
WHERE NOT (EXISTS (SELECT u.univ_id 
FROM student AS st 
WHERE u.univ_id = st.univ_id)) ORDER BY count(surname) DESC

13


[(10, 'ВГУ', 16, 'Студенты есть'),
 (22, 'МГУ', 10, 'Студенты есть'),
 (44, 'ФинУ', 10, 'Студенты есть'),
 (14, 'БГУ', 9, 'Студенты есть'),
 (46, 'ФГБОУ "Политех"', 9, 'Студенты есть'),
 (18, 'ВГМА', 7, 'Студенты есть'),
 (32, 'РГУ', 7, 'Студенты есть'),
 (47, 'КФУ', 6, 'Студенты есть'),
 (45, 'МГТУ им. Баумана', 5, 'Студенты есть'),
 (48, 'УЛГУ', 5, 'Студенты есть')]

In [27]:
# 9. Напишите запрос для получения полного списка университетов вместе с фамилиями преподавателей, в них работающих. 
# Для университетов, не имеющих преподавателей, поместите в список фразу ‘Преподавателей нет’. 
# результат должен содержать название университета, фамилию и имя преподавателя. 
# отсортируйте результат по фамилии и имени преподавателя.


sq = select(u.c.univ_id)
sq = sq.where(u.c.univ_id == l.c.univ_id).exists() 

s1 = select(u.c.univ_id, u.c.univ_name, l.c.surname, l.c.name)
s1 = s1.join(l)

s2 = select(u.c.univ_id, u.c.univ_name, text("'Преподавателей нет' AS Status"), text('NULL'))
s2 = s2.where(~sq)


stmt = union(s1, s2)
stmt = stmt.order_by(l.c.surname, l.c.name)


print_result(stmt)

SELECT u.univ_id, u.univ_name, l.surname, l.name 
FROM university AS u JOIN lecturer AS l ON u.univ_id = l.univ_id UNION SELECT u.univ_id, u.univ_name, 'Преподавателей нет' AS Status, NULL 
FROM university AS u 
WHERE NOT (EXISTS (SELECT u.univ_id 
FROM lecturer AS l 
WHERE u.univ_id = l.univ_id)) ORDER BY surname, name

32


[(10, 'ВГУ', 'Алексеев', 'Елисей'),
 (18, 'ВГМА', 'Артемьева', 'Майя'),
 (32, 'РГУ', 'Беляков', 'Мефодий'),
 (15, 'ТГУ', 'Беспалов', 'Юлиан'),
 (18, 'ВГМА', 'Богданова', 'Оксана'),
 (48, 'УЛГУ', 'Брагина', 'Нинель'),
 (45, 'МГТУ им. Баумана', 'Волков', 'Михаил'),
 (48, 'УЛГУ', 'Дмитриева', 'Оксана'),
 (32, 'РГУ', 'Дорофеев', 'Прокофий'),
 (48, 'УЛГУ', 'Елисеев', 'Виссарион')]

In [28]:
# 10. Выведите полный список студентов вместе с оценками, полученными ими на экзаменах. 
# Для студентов, не сдававших экзамены, в поле оценки поместите 0.


s1 = select(em.c.student_id)
s1 = s1.where(em.c.student_id == st.c.student_id).exists() # Сдавал что-то


s4 = select(text("'Не сдавал'"), st.c.student_id, st.c.surname, st.c.name, text("0"))
s4 = s4.where(~s1)

s5 = select(text("'Сдавал'"), st.c.student_id, st.c.surname, st.c.name, em.c.mark)
s5 = s5.where(em.c.mark != "None")
s5 = s5.join(em)


stmt = union(s5, s4)


print_result(stmt)

SELECT 'Сдавал', st.student_id, st.surname, st.name, em.mark 
FROM student AS st JOIN exam_marks AS em ON st.student_id = em.student_id 
WHERE em.mark != :mark_1 UNION SELECT 'Не сдавал', st.student_id, st.surname, st.name, 0 
FROM student AS st 
WHERE NOT (EXISTS (SELECT em.student_id 
FROM exam_marks AS em 
WHERE em.student_id = st.student_id))

97


[('Не сдавал', 3, 'Петров', 'Петр', 0),
 ('Не сдавал', 56, 'Дуб', 'Елизавета', 0),
 ('Не сдавал', 57, 'Кудряшова', 'Вера', 0),
 ('Не сдавал', 58, 'Журавлева', 'Вера', 0),
 ('Не сдавал', 60, 'Рожкова', 'Пелагея', 0),
 ('Не сдавал', 61, 'Соколова', 'Ия', 0),
 ('Не сдавал', 66, 'Некрасова', 'Вероника', 0),
 ('Не сдавал', 68, 'Казакова', 'Ольга', 0),
 ('Не сдавал', 69, 'Шубина', 'Елена', 0),
 ('Не сдавал', 70, 'Миронова', 'Анна', 0)]